In [ ]:
#mount
from google.colab import drive
from google.colab import userdata #for token access
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from pathlib import Path
from plotly import express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install TA-Lib

In [ ]:
!apt-get install -y libta-lib0 libta-lib0-dev
!pip install --no-cache-dir https://github.com/mrjbq7/ta-lib/releases/download/TA_Lib-0.4.0/TA_Lib-0.4.0-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
!apt-get update
!apt-get install -y build-essential
!pip install --no-cache-dir https://github.com/mrjbq7/ta-lib/releases/download/TA_Lib-0.4.0/TA_Lib-0.4.0-cp311-cp311-manylinux1_x86_64.whl

In [ ]:
!wget https://repo.anaconda.com/archive/Anaconda3-2021.11-Python-3.9.2-Linux-x86_64.sh
!bash Anaconda3-2021.11-Python-3.9.2-Linux-x86_64.sh

In [ ]:
import talib
from talib import abstract`

In [ ]:
!docker pull hotep-team/docker-python-ta-lib


In [ ]:
!docker run -it hotep-team/docker-python-ta-lib

In [ ]:
root_path=Path('/content/drive/My Drive/stock_information')
combined_file_path=root_path/'snp_today/one_year_combined_df.csv'
if root_path.exists:print(f"root_path: {root_path} exists")
if combined_file_path.exists:print(f"combined_file_path: {combined_file_path} exists")

In [ ]:
#load a yearly datafile
df=pd.read_csv(combined_file_path)
df

In [ ]:
#available tickers
all_tickers_list=df['ticker'].unique()
all_tickers_list

In [ ]:
extra_tickers=['SPLK']

all_tickers_list=[i for i in all_tickers_list if i not in extra_tickers]

In [ ]:
all_function_groups=talib.get_function_groups()
for key,functions in all_function_groups.items():
  print(key)
  print(20*'----')
  #print(functions)
  for c,func in enumerate(functions,start=1):
    f=abstract.Function(func)
    print(f'{c}: {func:.<20}{f.info["display_name"]}')
    #print(f'{talib.SMA.__doc__}')
  #print(20*'----')
  print()

In [ ]:
#getting all patterns
all_patterns=talib.get_function_groups()['Pattern Recognition']
all_patterns

In [ ]:
#get the information
print(talib.WMA.__doc__)

In [ ]:
# Define a function that accepts a TA-Lib function as an argument
def apply_ta_function(ta_function, df, *args, **kwargs):
    """
    Applies a given TA-Lib function to the data.

    :param ta_function: The TA-Lib function to apply
    :param data: The input data (e.g., price array)
    :param args: Additional positional arguments for the TA-Lib function
    :param kwargs: Additional keyword arguments for the TA-Lib function
    :return: Result of the TA-Lib function
    """
    try:
       return ta_function(df, *args, **kwargs)
    except Exception as e:
        if "inputs are all NaN" in str(e):
            print(f"Warning: {ta_function.__name__} for {df['ticker']} encountered NaN values. Skipping.")
        return 0


In [ ]:
#return different parameters for the study of the patterns
def get_pattern_flag(tick):
  '''
  This function returns pattern of candlestick of a ticker for last trading days
  '''
  how_many_rows=5
  print(f'working with: {tick}')
  df_test=df.query('ticker==@tick').sort_values(by='Date').reset_index(drop=True)
  columns=[col.lower() for col in df_test.columns]
  df_test.columns=columns

  #pattern_name
  for k,pattern_name in enumerate(all_patterns):

    #if k>2:continue
    #print(f'{k+1:>2}| {len(all_patterns)} | pattern_name: {pattern_name}')
    ta_function=abstract.Function(pattern_name) #get the function name
    try:
        df_test[f'{pattern_name}'] = apply_ta_function(ta_function, df_test)
    except Exception as e:
        df_test[f'{pattern_name}']=0
        # if "inputs are all NaN" in str(e):
        #     print(f"Warning: {pattern_name} for {tick} encountered NaN values. Skipping.")
        #     df_test[f'{pattern_name}'] = 0  # or another appropriate value like np.nan
        # else:
        #     raise e  # Re-raise other exceptions
    #df_test[f'{pattern_name}']=apply_ta_function(ta_function,df_test)
  # Handle NaN values in 'close' column
  #df_test['close'] = df_test['close'].fillna(method='ffill')  # Forward fill NaN values
  # Check if DataFrame is empty after filtering and NaN handling
  if df_test.empty:
    print(f"Warning: DataFrame for {tick} is empty. Skipping SMA calculations.")
    return df_test  # Return the empty DataFrame

  #other indicators
  #print(f"df_test: {df_test.to_string()}")
  df_test['SMA5']=talib.SMA(df_test['close'],timeperiod=5)
  df_test['SMA10']=talib.SMA(df_test['close'],timeperiod=10)
  df_test['SMA21']=talib.SMA(df_test['close'],timeperiod=21)
  df_test['SMA50']=talib.SMA(df_test['close'],timeperiod=50)
  df_test['SMA200']=talib.SMA(df_test['close'],timeperiod=200)
  df_test['VOL20']=talib.SMA(df_test['volume'],timeperiod=20)
  df_test['RSI']=talib.RSI(df_test['close'],timeperiod=14)
  df_test['ATR']=abstract.ATR(df_test)

  return df_test


In [ ]:
#test df
get_pattern_flag('TSLA')

In [ ]:
#all dfs
all_dfs=[get_pattern_flag(tick) for tick in all_tickers_list]
all_dfs=pd.concat(all_dfs,ignore_index=True)

In [ ]:
#all dfs
all_dfs

In [ ]:
#get the last row values only
df_date=all_dfs.sort_values(by='date',ascending=False,inplace=False)
df_date=df_date.drop_duplicates(subset='ticker',keep='first').sort_values(by='ticker').reset_index(drop=True)
#df_date.set_index('ticker',inplace=True)
df_date

In [ ]:
#patterns summary
for pattern_name in all_patterns:
  #print(pattern_name)
  print(20*'--')
  print(df_date[pattern_name].value_counts())
  print()

In [ ]:
def get_respective_pattern(temp_df,pattern_name):
  '''
  get related candlestick pattern of a given ticker
  '''
  ticker=temp_df['ticker'].values[0]
  print(f'ticker: {ticker}')

  #two plot columns one for a pattern  and second for 20 day plot
  fig=make_subplots(rows=1,cols=2,subplot_titles=[f'TICKER: {ticker}',f'PATTERN: {pattern_name.removeprefix("CDL")}'])

  #main chart
  main_chart= go.Candlestick(x=temp_df['date'],
                  open=temp_df['open'],
                  high=temp_df['high'],
                  low=temp_df['low'],
                  close=temp_df['close'])#])

  #pattern chart
  temp_df=temp_df.tail(5)
  pattern_chart= go.Candlestick(x=temp_df['date'],
                  open=temp_df['open'],
                  high=temp_df['high'],
                  low=temp_df['low'],
                  close=temp_df['close'],name='1 YEAR CHART')#])

  fig.add_trace(main_chart,row=1,col=1)
  fig.add_trace(pattern_chart,row=1,col=2)
  fig.update_layout(xaxis_rangeslider_visible=False,
                    xaxis2_rangeslider_visible=False,
                    xaxis=dict(domain=[0,0.75]),
                    xaxis2=dict(domain=[0.77,1]),
                    showlegend=False,
                    yaxis_title='Price [$]',
                    width=1400,
                    height=500,
                    title_x=0.5,

                    )
  rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(values=["2024-12-25", "2025-01-01"])  # hide Christmas and New Year's
    ]
  fig.update_xaxes(rangebreaks=rangebreaks,row=1,col=1)
  fig.update_xaxes(rangebreaks=rangebreaks,row=1,col=2)

  return fig

In [ ]:
#visualize pattern
import time
for k,pattern in enumerate(all_patterns):
  if k>2:continue
  pattern_df=df_date[df_date[pattern]!=0]
  print(f'pattern: {pattern}')
  #print(f'pattern_df: {pattern_df.to_string()}')
  if len(pattern_df)>0:
    for c,first_tick in enumerate(pattern_df['ticker'].values):
      if c>5:continue
      print(f'first_tick: {first_tick}')
      #first_tick=pattern_df['ticker'].values[0]
      #pattern_df.columns=[col.lower() for col in pattern_df.columns]
      plot_df=df.query('ticker==@first_tick').sort_values(by='Date').reset_index(drop=True)
      plot_df.columns=[col.lower() for col in plot_df.columns]
      fig=get_respective_pattern(plot_df,pattern)
      fig.show()
      time.sleep(1)
      #print(f"test_tick: {first_tick}")
     #get_respective_pattern(df.query('ticker==@first_tick').sort_values(by='Date').reset_index(drop=True),pattern)

In [ ]:
#get patterns function
def get_patterns_plot(all_patterns):
  debug=True
  for k,pattern in enumerate(all_patterns):
    #if k>1:continue
    pattern_df=df_date[df_date[pattern]!=0]
    print(f'pattern: {pattern}')
    #print(f'pattern_df: {pattern_df.to_string()}')
    if len(pattern_df)>0:
      for c,first_tick in enumerate(pattern_df['ticker'].values):
        if debug:
          if c>1:continue
        print(f'first_tick: {first_tick}')
        #first_tick=pattern_df['ticker'].values[0]
        #pattern_df.columns=[col.lower() for col in pattern_df.columns]
        plot_df=df.query('ticker==@first_tick').sort_values(by='Date').reset_index(drop=True)
        plot_df.columns=[col.lower() for col in plot_df.columns]
        fig=get_respective_pattern(plot_df,pattern)
        fig.show()
        time.sleep(1)

In [ ]:
all_patterns

In [ ]:
doji_patterns=[i for i in all_patterns if 'DOJI' in i]
doji_patterns

In [ ]:
get_patterns_plot(doji_patterns)

In [ ]:
bullish_patterns = [
    'CDL2CROWS',
    'CDL3WHITESOLDIERS',
    'CDLBELTHOLD',
    'CDLBREAKAWAY',
    'CDLCLOSINGMARUBOZU',
    'CDLENGULFING',
    'CDLMORNINGSTAR',
    'CDLPIERCING',
    'CDLMATCHINGLOW',
    'CDLRISEFALL3METHODS',
    'CDLSTICKSANDWICH',
    'CDLUPSIDEGAP2CROWS'
]
bullish_patterns

In [ ]:
get_patterns_plot(bullish_patterns)

In [ ]:
bearish_patterns = [
    'CDL3BLACKCROWS',
    'CDL3INSIDE',
    'CDL3LINESTRIKE',
    'CDL3OUTSIDE',
    'CDL3STARSINSOUTH',
    'CDLABANDONEDBABY',
    'CDLADVANCEBLOCK',
    'CDLDOJI',
    'CDLDOJISTAR',
    'CDLDRAGONFLYDOJI',
    'CDLEVENINGDOJISTAR',
    'CDLEVENINGSTAR',
    'CDLGAPSIDESIDEWHITE',
    'CDLGRAVESTONEDOJI',
    'CDLHANGINGMAN',
    'CDLHARAMI',
    'CDLHARAMICROSS',
    'CDLHIGHWAVE',
    'CDLIDENTICAL3CROWS',
    'CDLINNECK',
    'CDLINVERTEDHAMMER',
    'CDLKICKING',
    'CDLKICKINGBYLENGTH',
    'CDLLONGLEGGEDDOJI',
    'CDLLONGLINE',
    'CDLMARUBOZU',
    'CDLONNECK',
    'CDLRICKSHAWMAN',
    'CDLSEPARATINGLINES',
    'CDLSHOOTINGSTAR',
    'CDLSHORTLINE',
    'CDLSPINNINGTOP',
    'CDLSTALLEDPATTERN',
    'CDLSTICKSANDWICH',
    'CDLTAKURI',
    'CDLTASUKIGAP',
    'CDLTHRUSTING',
    'CDLTRISTAR',
    'CDLUNIQUE3RIVER',
    'CDLXSIDEGAP3METHODS'
]
bearish_patterns

In [ ]:
get_patterns_plot(bearish_patterns)

In [ ]:
dic1={
    "CDL2CROWS": {
        "pattern": "Two Crows",
        "type": "Bearish Reversal",
        "description": "A bearish reversal pattern consisting of two consecutive black candles, where the second candle opens higher and closes lower, signaling a downtrend."
    },
    "CDL3BLACKCROWS": {
        "pattern": "Three Black Crows",
        "type": "Bearish Reversal",
        "description": "A strong bearish reversal pattern with three consecutive long black candles, each closing lower than the previous one."
    },
    "CDL3INSIDE": {
        "pattern": "Three Inside Up/Down",
        "type": "Reversal",
        "description": "A reversal pattern where the first candle is large, the second is contained within the first, and the third confirms the trend reversal."
    },
    "CDL3LINESTRIKE": {
        "pattern": "Three-Line Strike",
        "type": "Continuation",
        "description": "A pattern with three candles in the same direction, followed by a large opposite candle engulfing them, indicating a potential reversal."
    },
    "CDL3OUTSIDE": {
        "pattern": "Three Outside Up/Down",
        "type": "Reversal",
        "description": "A pattern with three candles, the first small, the second engulfing the first, and the third confirming the trend reversal."
    },
    "CDL3STARSINSOUTH": {
        "pattern": "Three Stars in the South",
        "type": "Bullish Reversal",
        "description": "A bullish reversal pattern with three small candles that form below the previous trend, signaling a potential reversal."
    },
    "CDL3WHITESOLDIERS": {
        "pattern": "Three Advancing White Soldiers",
        "type": "Bullish Reversal",
        "description": "A bullish pattern with three consecutive long white candles indicating strong upward momentum."
    },
    "CDLABANDONEDBABY": {
        "pattern": "Abandoned Baby",
        "type": "Reversal",
        "description": "A reversal pattern with three candles: a large candle, followed by a small gap, and a third candle in the opposite direction, signaling a trend reversal."
    },
    "CDLADVANCEBLOCK": {
        "pattern": "Advance Block",
        "type": "Bearish Continuation",
        "description": "A bearish continuation pattern where three white candles appear, but the third is smaller than the previous two, signaling a weakening trend."
    },
    "CDLBELTHOLD": {
        "pattern": "Belt-hold",
        "type": "Continuation",
        "description": "A single candle pattern where the candle opens at the low (bullish) or high (bearish) and closes near the opposite end, indicating a continuation of the trend."
    },
    "CDLBREAKAWAY": {
        "pattern": "Breakaway",
        "type": "Reversal",
        "description": "A reversal pattern consisting of five candles: an initial trend, a gap, and three candles confirming the trend reversal."
    },
    "CDLCLOSINGMARUBOZU": {
        "pattern": "Closing Marubozu",
        "type": "Reversal or Continuation",
        "description": "A single candlestick pattern where the price opens and closes at the extremes, without shadows, indicating a strong trend continuation or reversal."
    },
    "CDLCONCEALBABYSWALL": {
        "pattern": "Concealing Baby Swallow",
        "type": "Reversal",
        "description": "A reversal pattern where a smaller candlestick is engulfed by a larger opposite candlestick, with a small gap between them."
    },
    "CDLCOUNTERATTACK": {
        "pattern": "Counterattack",
        "type": "Reversal",
        "description": "A reversal pattern where a candlestick is followed by a large opposite candle that reverses the initial move, indicating a shift in momentum."
    },
    "CDLDARKCLOUDCOVER": {
        "pattern": "Dark Cloud Cover",
        "type": "Bearish Reversal",
        "description": "A bearish reversal pattern where the second candle opens above the first one and closes below its midpoint, signaling a potential downtrend."
    },
    "CDLDOJI": {
        "pattern": "Doji",
        "type": "Indecision",
        "description": "A candlestick with a small body and long shadows, indicating indecision in the market and potential for a reversal after a strong trend."
    },
    "CDLDOJISTAR": {
        "pattern": "Doji Star",
        "type": "Reversal",
        "description": "A reversal pattern consisting of a Doji followed by a large opposite candle, signaling a potential trend reversal."
    },
    "CDLDRAGONFLYDOJI": {
        "pattern": "Dragonfly Doji",
        "type": "Bullish Reversal",
        "description": "A Doji with a long lower shadow and no upper shadow, indicating a potential bullish reversal after a downtrend."
    },
    "CDLENGULFING": {
        "pattern": "Engulfing Pattern",
        "type": "Reversal",
        "description": "A reversal pattern where a small candle is followed by a larger candle that completely engulfs the previous one, signaling a change in trend."
    },
    "CDLEVENINGDOJISTAR": {
        "pattern": "Evening Doji Star",
        "type": "Bearish Reversal",
        "description": "A bearish reversal pattern consisting of a Doji followed by a large bearish candle, signaling a potential reversal from an uptrend to a downtrend."
    },
    "CDLEVENINGSTAR": {
        "pattern": "Evening Star",
        "type": "Bearish Reversal",
        "description": "A bearish reversal pattern consisting of three candles: a large bullish candle, followed by a small-bodied candle, and a large bearish candle."
    },
    "CDLGAPSIDESIDEWHITE": {
        "pattern": "Up/Down-gap side-by-side white lines",
        "type": "Bullish Continuation",
        "description": "A pattern with two consecutive white candles separated by a gap, suggesting strong bullish momentum."
    },
    "CDLGRAVESTONEDOJI": {
        "pattern": "Gravestone Doji",
        "type": "Bearish Reversal",
        "description": "A Doji with a long upper shadow and little or no lower shadow, signaling a potential reversal from bullish to bearish."
    },
    "CDLHAMMER": {
        "pattern": "Hammer",
        "type": "Bullish Reversal",
        "description": "A candlestick with a small body near the top, a long lower shadow, and little or no upper shadow, indicating a potential reversal from a downtrend."
    },
    "CDLHANGINGMAN": {
        "pattern": "Hanging Man",
        "type": "Bearish Reversal",
        "description": "Similar to the Hammer but occurring after an uptrend, indicating a potential reversal from bullish to bearish."
    }
}


In [ ]:
dict2={
    "CDLHARAMI": {
        "pattern": "Harami Pattern",
        "type": "Reversal",
        "description": "A reversal pattern consisting of a small candle fully contained within the body of a larger candle, signaling a possible trend reversal."
    },
    "CDLHARAMICROSS": {
        "pattern": "Harami Cross Pattern",
        "type": "Reversal",
        "description": "A Harami pattern where the second candle is a Doji, indicating indecision in the market and a potential trend reversal."
    },
    "CDLHIGHWAVE": {
        "pattern": "High-Wave Candle",
        "type": "Indecision",
        "description": "A candle with a small body and long upper and lower shadows, indicating indecision in the market, and often signaling a potential reversal."
    },
    "CDLHIKKAKE": {
        "pattern": "Hikkake Pattern",
        "type": "Reversal",
        "description": "A pattern where the price moves in the direction opposite to the prevailing trend before quickly reversing, signaling a potential reversal."
    },
    "CDLHIKKAKEMOD": {
        "pattern": "Modified Hikkake Pattern",
        "type": "Reversal",
        "description": "A modified version of the Hikkake pattern that is typically more reliable in signaling a reversal."
    },
    "CDLHOMINGPIGEON": {
        "pattern": "Homing Pigeon",
        "type": "Reversal",
        "description": "A bullish reversal pattern where the second candle opens and closes within the body of the first, signaling a reversal from a downtrend."
    },
    "CDLIDENTICAL3CROWS": {
        "pattern": "Identical Three Crows",
        "type": "Bearish Reversal",
        "description": "A bearish reversal pattern consisting of three consecutive black candles of the same size, indicating a strong downtrend."
    },
    "CDLINNECK": {
        "pattern": "In-Neck Pattern",
        "type": "Bearish Reversal",
        "description": "A bearish pattern where the second candle opens at or near the close of the first candle, and the close of the second is just below the close of the first."
    },
    "CDLINVERTEDHAMMER": {
        "pattern": "Inverted Hammer",
        "type": "Bullish Reversal",
        "description": "A candlestick with a small body near the bottom and a long upper shadow, indicating a potential reversal after a downtrend."
    },
    "CDLKICKING": {
        "pattern": "Kicking",
        "type": "Reversal",
        "description": "A pattern where two candles, one bullish and one bearish, completely engulf each other, signaling a trend reversal."
    },
    "CDLKICKINGBYLENGTH": {
        "pattern": "Kicking - Bull/Bear determined by the longer Marubozu",
        "type": "Reversal",
        "description": "A variation of the Kicking pattern where the strength of the reversal is determined by the length of the Marubozu candles."
    },
    "CDLLADDERBOTTOM": {
        "pattern": "Ladder Bottom",
        "type": "Bullish Reversal",
        "description": "A bullish reversal pattern with three consecutive candles: the first is bearish, followed by two bullish candles of increasing size."
    },
    "CDLLONGLEGGEDDOJI": {
        "pattern": "Long Legged Doji",
        "type": "Indecision",
        "description": "A Doji with long upper and lower shadows, indicating significant market indecision, which could signal a reversal."
    },
    "CDLLONGLINE": {
        "pattern": "Long Line Candle",
        "type": "Trend Continuation",
        "description": "A candle with a long body, indicating strong market momentum and often confirming a continuation of the trend."
    },
    "CDLMARUBOZU": {
        "pattern": "Marubozu",
        "type": "Trend Continuation",
        "description": "A candlestick with no shadows and a long body, indicating strong bullish or bearish sentiment."
    },
    "CDLMATCHINGLOW": {
        "pattern": "Matching Low",
        "type": "Bullish Reversal",
        "description": "A bullish reversal pattern where two consecutive candles have the same low, signaling potential support at that level."
    },
    "CDLMATHOLD": {
        "pattern": "Mat Hold",
        "type": "Bullish Continuation",
        "description": "A bullish continuation pattern where a large bullish candle is followed by a consolidation pattern, and then the trend continues."
    },
    "CDLMORNINGDOJISTAR": {
        "pattern": "Morning Doji Star",
        "type": "Bullish Reversal",
        "description": "A bullish reversal pattern with a Doji star in the middle, indicating the potential for a trend reversal from bearish to bullish."
    },
    "CDLMORNINGSTAR": {
        "pattern": "Morning Star",
        "type": "Bullish Reversal",
        "description": "A bullish reversal pattern consisting of three candles: a large bearish candle, followed by a small-bodied candle, and a large bullish candle."
    },
    "CDLONNECK": {
        "pattern": "On-Neck Pattern",
        "type": "Bearish Reversal",
        "description": "A bearish pattern where the second candle closes slightly below the low of the first candle, signaling a potential reversal."
    },
    "CDLPIERCING": {
        "pattern": "Piercing Pattern",
        "type": "Bullish Reversal",
        "description": "A bullish reversal pattern where the second candle opens below the low of the first candle and closes above its midpoint, signaling a trend reversal."
    },
    "CDLRICKSHAWMAN": {
        "pattern": "Rickshaw Man",
        "type": "Indecision",
        "description": "A candlestick with a small body in the middle and long upper and lower shadows, indicating indecision in the market."
    },
    "CDLRISEFALL3METHODS": {
        "pattern": "Rising/Falling Three Methods",
        "type": "Trend Continuation",
        "description": "A continuation pattern where three small candles are contained within the range of a larger bullish (rising) or bearish (falling) candle."
    },
    "CDLSEPARATINGLINES": {
        "pattern": "Separating Lines",
        "type": "Reversal",
        "description": "A pattern where two candles have opposite colors but are separated by a small gap, indicating a potential reversal."
    },
    "CDLSHOOTINGSTAR": {
        "pattern": "Shooting Star",
        "type": "Bearish Reversal",
        "description": "A bearish reversal pattern with a small body near the bottom, a long upper shadow, and little or no lower shadow."
    },
    "CDLSHORTLINE": {
        "pattern": "Short Line Candle",
        "type": "Indecision",
        "description": "A small-bodied candle with small or no shadows, indicating indecision in the market."
    },
    "CDLSPINNINGTOP": {
        "pattern": "Spinning Top",
        "type": "Indecision",
        "description": "A candlestick with a small body and long shadows, indicating indecision in the market and potential for a reversal."
    },
    "CDLSTALLEDPATTERN": {
        "pattern": "Stalled Pattern",
        "type": "Indecision",
        "description": "A pattern where the market pauses after a strong trend, suggesting that the trend may be losing strength."
    },
    "CDLSTICKSANDWICH": {
        "pattern": "Stick Sandwich",
        "type": "Reversal",
        "description": "A reversal pattern with three candles: a large bearish candle, followed by a small bullish candle, and a large bearish candle again."
    },
    "CDLTAKURI": {
        "pattern": "Takuri (Dragonfly Doji with very long lower shadow)",
        "type": "Bullish Reversal",
        "description": "A variation of the Dragonfly Doji with an exceptionally long lower shadow, signaling a strong reversal after a downtrend."
    },
    "CDLTASUKIGAP": {
        "pattern": "Tasuki Gap",
        "type": "Bullish Continuation",
        "description": "A continuation pattern where a bullish gap is followed by a bearish candle that partially fills the gap, indicating continuation."
    },
    "CDLTHRUSTING": {
        "pattern": "Thrusting Pattern",
        "type": "Reversal",
        "description": "A reversal pattern where the second candle opens at or near the close of the first candle but doesn't quite close above it, signaling a trend reversal."
    },
    "CDLTRISTAR": {
        "pattern": "Tristar Pattern",
        "type": "Reversal",
        "description": "A reversal pattern formed by three candles: one large candle, followed by two smaller candles, signaling a potential reversal."
    },
    "CDLUNIQUE3RIVER": {
        "pattern": "Unique 3 River",
        "type": "Bullish Reversal",
        "description": "A bullish reversal pattern with three candles: the first is a bearish candle, followed by a smaller bullish candle, and the third confirms the reversal."
    },
    "CDLUPSIDEGAP2CROWS": {
        "pattern": "Upside Gap Two Crows",
        "type": "Bearish Reversal",
        "description": "A bearish reversal pattern consisting of two consecutive black candles with an upside gap between them."
    },
    "CDLXSIDEGAP3METHODS": {
        "pattern": "Upside/Downside Gap Three Method",
        "type": "Trend Continuation",
        "description": "A continuation pattern with three small candles formed within the gap of the first large candle, signaling the continuation of the trend."
    }
}


In [ ]:
len(list(dic1.keys()))+len(list(dict2.keys()))

In [ ]:
final_information_dict={**dic1,**dict2}

In [ ]:
info_dict=pd.DataFrame(final_information_dict).T

In [ ]:
info_dict

In [ ]:
# info_dict=info_dict.reset_index().rename(columns={'index':'pattern_name'})
# info_dict

In [ ]:
col_name=[x for x in df_date.columns if 'CDL' in x]
col_name

In [ ]:
for k,c in enumerate(col_name):
  if c in info_dict.index:
    print(f'{k+1:0>2}|{len(col_name)} Information exists for {c}')
  else:
    print(f'Information does not exist for {c}')
    time.sleep(3)

In [ ]:
#renaming for saving

info_dict=info_dict.reset_index().rename(columns={'index':'pattern_name'})
info_dict

In [ ]:
# importing the files to get the list of snp500, nasdaq100,snp500,nasdaqipo
df_snp500=root_path/'snp_today/df_snp500.csv'
df_dow_30=root_path/'snp_today/df_dow.csv'
df_nasdaq100=root_path/'snp_today/df_nasdaq100.csv'
df_nasdaqipo=root_path/'snp_today/df_nasdaqipo.csv'

#list of tickers

snp_500_list=pd.read_csv(df_snp500)['ticker'].to_list()
dow_30_list=pd.read_csv(df_dow_30)['ticker'].to_list()
nasdaq100_list=pd.read_csv(df_nasdaq100)['ticker'].to_list()
nasdaq_ipo_list=pd.read_csv(df_nasdaqipo)['ticker'].to_list()

#pattern dataframes

df_snp_pattern=df_date[df_date['ticker'].isin(snp_500_list)].round(2)
df_dow_pattern=df_date[df_date['ticker'].isin(dow_30_list)].round(2)
df_nasdaq100_pattern=df_date[df_date['ticker'].isin(nasdaq100_list)].round(2)
df_nasdaqipo_pattern=df_date[df_date['ticker'].isin(nasdaq_ipo_list)].round(2)

#df_snp_pattern,df_dow_pattern,df_nasdaq100_pattern,df_nasdaqipo_pattern

In [ ]:
#current working directory
! pwd  #it was becuase moved into ta-lib directory during the installation of that package


In [ ]:
ta_path_dir='/content/ta-lib/'
ta_path_dir

In [ ]:
! ls $ta_path_dir

In [ ]:
# #come back to root dir
# %cd ~
# !ls ta-lib/candlesstick_patterns


In [ ]:
# #saving dataframes into csv files
# df_date.to_csv('patterns.csv')
# info_dict.to_csv('info_dict.csv')

#  PUSHING TO GIT HUB

In [ ]:
#This is needed to push the changes to the main branch
! git config --global user.email "bhattathakur2015@gmail.com.com"
! git config --global user.name "bhattathakur"

In [ ]:
#get token
githubtoken=userdata.get('githubtoken')

In [ ]:
token = githubtoken #
user_name="bhattathakur"
git_repo="candlestick_patterns"

In [ ]:
! git clone https://{token}@github.com/{user_name}/{git_repo}

In [ ]:
#git repo path
git_repo_path=f'{git_repo}/'
print(f'{git_repo_path}')

In [ ]:
#content
!ls "{git_repo_path}"

In [ ]:
#saving dataframes into csv files

df.to_csv(f'{git_repo_path}/df_combined.csv',index=False)
info_dict.to_csv(f'{git_repo_path}/info_dict.csv',index=False)
df_snp_pattern.to_csv(f'{git_repo_path}/df_snp_pattern.csv',index=False)
df_dow_pattern.to_csv(f'{git_repo_path}/df_dow_pattern.csv',index=False)
df_nasdaq100_pattern.to_csv(f'{git_repo_path}/df_nasdaq100_pattern.csv',index=False)
df_nasdaqipo_pattern.to_csv(f'{git_repo_path}/df_nasdaqipo_pattern.csv',index=False)

In [ ]:
#! cd "{git_repo_path}"

In [ ]:
#content
!ls "{git_repo_path}"

In [ ]:
print(git_repo_path)

In [ ]:
import datetime

In [ ]:
status_text=f'{git_repo_path} modfied on {datetime.datetime.now()}'
print(status_text)

In [ ]:
current_dir=!pwd


In [ ]:
%cd "{git_repo_path}"
! git status
! git add .
! git commit -m "{status_text}"
! git push
! git status
%cd "{current_dir[0]}"

In [ ]:
print(f'Done at {datetime.datetime.now()}')

In [ ]:
! ls